# Polyminoes

More powerful version of [Pentomino](./Pentomino.ipynb). Especially:

1. Support diversified grid (via [Puzzle Tools](https://smilingwayne.github.io/PuzzleTools/)). Grid can be found at [default_json](../assets/data/Polyminoes/default_shapes.json).
2. Support x-mino(es) from 1 to 5, including 1 monomino, 1 domino, 2 trominoes, 5 tetrominoes, and 12 pentominoes, for a total of 89 squares.


## Dependencies

In [ ]:
import numpy as np
from ortools.sat.python import cp_model as cp
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.pyplot import MultipleLocator
import random
from collections import defaultdict

## Pattern Rotations

In [ ]:
COLORS = {
    "F" : "#8CD4C7",
    "I" : "#FFFFB4",
    "L" : "#BFBADA",
    "N" : "#BD7FBD",
    "P" : "#D9D9D9",
    "T" : "#FCB561",
    "U" : "#B3DE68",
    "V" : "#FCCCE5",
    "W" : "#FA7F73",
    "X" : "#7EB0D4",
    "Y" : "#CCEBC4",
    "Z" : "#FFED70"
}

def all_possible_rotations(shape, name):
    # shape = [(0,0), (0,1), (0,2), ...] 
    all_patterns = []
    pattern_index = []
    
    max_x = max([coord[0] for coord in shape])
    max_y = max([coord[1] for coord in shape])

    new_grid = [[0] * (max_y + 1) for _ in range(max_x + 1)] 
    for (a, b) in shape: 
        new_grid[a][b] = 1 
    pattern = np.array(new_grid) 
    cnt_idx = 0
    for i in range(1, 5):
        pattern_rot = np.rot90(pattern,  k = i, axes = (0,1))
        pattern_flip = np.fliplr(pattern_rot)
        pattern_rot_flag = True 
        pattern_flip_flag = True
        for compare_pattern in all_patterns:
            if pattern_rot_flag and np.array_equal(pattern_rot, compare_pattern):
                pattern_rot_flag = False 
            if pattern_flip_flag and np.array_equal(pattern_flip, compare_pattern):
                pattern_flip_flag = False
            if not pattern_rot_flag and not pattern_flip_flag:
                break
            
        if pattern_rot_flag:
            cnt_idx += 1
            all_patterns.append(pattern_rot)
            pattern_index.append(f"{name}_{cnt_idx}")
        
        if pattern_flip_flag and not np.array_equal(pattern_rot, pattern_flip):
            cnt_idx += 1
            all_patterns.append(pattern_flip)
            pattern_index.append(f"{name}_{cnt_idx}")

    def extract_coordinates(array):
        coordinates = []
        rows, cols = array.shape
        
        for row in range(rows):
            for col in range(cols):
                if array[row, col] == 1:
                    x = col
                    y = rows - 1 - row  # 计算坐标的 y 值
                    coordinates.append((x, y))
        
        return coordinates
    all_patterns = list(map(extract_coordinates, all_patterns.copy()))
    res_all_patterns = dict(zip(pattern_index, all_patterns))
    # print(f"# of Pentomino in Total: {len(all_patterns)}")
    return res_all_patterns

if __name__ == "__main__": 
    test_pattern =  all_possible_rotations([(1,0), (1,1), (1,2), (0,1), (2, 2), (2, 3)], name = 'T')
    print(test_pattern)

{'T_1': [(2, 3), (1, 2), (2, 2), (0, 1), (1, 1), (1, 0)], 'T_2': [(0, 3), (0, 2), (1, 2), (1, 1), (2, 1), (1, 0)], 'T_3': [(0, 2), (1, 2), (1, 1), (2, 1), (3, 1), (2, 0)], 'T_4': [(2, 2), (3, 2), (0, 1), (1, 1), (2, 1), (1, 0)], 'T_5': [(1, 3), (1, 2), (2, 2), (0, 1), (1, 1), (0, 0)], 'T_6': [(1, 3), (0, 2), (1, 2), (1, 1), (2, 1), (2, 0)], 'T_7': [(1, 2), (0, 1), (1, 1), (2, 1), (2, 0), (3, 0)], 'T_8': [(2, 2), (1, 1), (2, 1), (3, 1), (0, 0), (1, 0)]}


## Pattern Generation

In [29]:
def get_polyminoes_by_size(sizes): 
    sizes = set(sizes)
    shapes = []
    shape_names = []
    for size in sizes: 
        if size == 5: 
            shapes += [
                [(0,1), (1,0), (1,1), (1,2), (2,2)], # F
                [(0,0), (1,0), (2,0), (3,0), (4,0)], # I
                [(0,0), (1,0), (2,0), (3,0), (3,1)], # L
                [(0,0), (1,0), (2,0), (2,1), (3,1)], # N 
                [(0,0), (1,0), (1,1), (2,0), (2,1)], # P 
                [(0,2), (1,0), (1,1), (1,2), (2,2)], # T
                [(0,0), (0,1), (1,0), (2,0), (2,1)], # U 
                [(0,0), (0,1), (0,2), (1,0), (2,0)], # V
                [(0,1), (0,2), (1,0), (1,1), (2,0)], # W
                [(0,1), (1,0), (1,1), (1,2), (2,1)], # X 
                [(0,1), (1,0), (1,1), (1,2), (1,3)], # Y
                [(0,2), (1,0), (1,1), (1,2), (2,0)]  # Z 
            ]
            shape_names += ['F', 'I', 'L', 'N', 'P', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
        if size == 4: 
            shapes += [
                [(0,0), (1,0), (2,0), (3,0)], # I4 
                [(0,0), (1,0), (2,0), (2,1)], # L4
                [(0,0), (0,1), (1,0), (1,1)], # O4
                [(0,0), (1,0), (1,1), (2,0)], # T4
                [(0,1), (1,0), (1,1), (2,0)]  # Z4
            ]
            shape_names += ['I4', 'L4', 'O4', 'T4', 'Z4']
        if size == 3: 
            shapes += [
                [(0, 0), (0, 1), (0, 2)], 
                [(0, 0), (0, 1), (1, 0)]
            ]
            shape_names += ["I3", "V3"]
        if size == 2:
            shapes += [
                [(0, 0), (0, 1)]
            ]
            shape_names += ["I2"]
        if size == 1:
            shapes += [
                [(0,0)]
            ]
            shape_names += ["O1"]
    return shapes, shape_names


def clear_padding(coords):
    min_x = min(x for x, _ in coords)
    min_y = min(y for _, y in coords)
    offset_x = min_x - 0 
    offset_y = min_y - 0 
    if offset_x > 0 or offset_y > 0:
        new_coords = []
        for (x, y) in coords: 
            new_coords.append((x - offset_x, y - offset_y))
        return new_coords 
    else:
        return coords

def generate_grid(name = 'default', raw_grid = None): 
    if name == "3x20": 
        return {
            "x": 20, 
            "y": 3,  
            "grid": ",".join([f"{a}_{b}" for a in range(20) for b in range(3)])
        }

    if name == "6x10":
        return {
            "x": 10, 
            "y": 6,  
            "grid": ",".join([f"{a}_{b}" for a in range(10) for b in range(6)])
        }

    if raw_grid: 
        raw_grid = clear_padding(raw_grid)
        max_x = 0
        max_y = 0
        for (x, y) in raw_grid:
            max_x = max(max_x, x)
            max_y = max(max_y, y)
        return {
            "x": max_x + 1, 
            "y": max_y + 1,
            "grid": ",".join([f"{x}_{y}" for (x, y) in raw_grid])
        }
    else:
        return {}


## Grid and solver

In [ ]:
class Grid:
    def __init__(self, grid_dict): 
        self.width = grid_dict['x']
        self.height = grid_dict['y']
        self.grid_str = grid_dict['grid']
        self.positions = set(tuple(int(part) for part in coord.split('_')) for coord in self.grid_str.split(','))
        self.cand_shapes = grid_dict['cand_shapes']
        self.model = cp.CpModel() 
        self.solver = cp.CpSolver()
        self.x = dict()  # variables
        self.avail_variables = defaultdict(list) 
        # record available variables of each cell
        self.shape_type = defaultdict(list)

    def get_all_feasible_pos(self, shape):
        candidates = set()
        for x_1 in range(self.width):
            for y_1 in range(self.height):
                if self.check_fit(shape, x_1, y_1):
                    candidates.add((x_1, y_1))
        return candidates

    def check_fit(self, shape, x_, y_): 
        for (x, y) in shape:
            if (x + x_, y + y_) not in self.positions:
                return False 
        return True

    def add_all_vars(self): 
        for shape_name, shape in self.cand_shapes.items(): 
            # print(shape_name, shape)
            shape_info = shape_name.split("_")
            temp_type, temp_index = shape_info[0], shape_info[1]
            cur_cand_pos = self.get_all_feasible_pos(shape) 
            
            for (x_1, y_1) in cur_cand_pos:
                self.x[f"{temp_type}_{temp_index}_{x_1}_{y_1}"] = self.model.NewBoolVar(f"{temp_type}_{temp_index}_{x_1}_{y_1}")
                for (x_2, y_2) in shape: 
                    # Exact Cover must start from the grid!
                    self.avail_variables[x_1 + x_2, y_1 + y_2].append(self.x[f"{temp_type}_{temp_index}_{x_1}_{y_1}"])
                self.shape_type[temp_type].append(self.x[f"{temp_type}_{temp_index}_{x_1}_{y_1}"])
        
        for temp_type, temp_vars in self.shape_type.items():
            self.model.Add(sum(temp_vars) == 1 )

        for k, avail_vars in self.avail_variables.items(): 
            # Adjust here!!!!
            if (int(k[0]), int(k[1])) in self.positions: 
                self.model.Add(sum(avail_vars) == 1) 
    
    def solve(self):
        self.add_all_vars()
        status = self.solver.Solve(self.model)
        result_coords = []
        result_shape_name = []
        if status == cp.OPTIMAL:
            print("Found!")
            for k_ in self.x.keys(): 
                if self.solver.Value(self.x[k_]) > 1e-5: 
                    shape_info = k_.split("_")
                    cur_shape = self.cand_shapes[f"{shape_info[0]}_{shape_info[1]}"]
                    cur_res = []
                    for (x_2, y_2) in cur_shape: 
                        cur_res.append([x_2 + int(shape_info[2]), y_2 + int(shape_info[3])])
                    result_coords.append(cur_res)
                    result_shape_name.append(f"{shape_info[0]}")
            print(result_coords)

        else:
            print("Can't find Optimal.")
        return result_coords, result_shape_name
            

if __name__ == "__main__":

    # raw_grid = classic_shapes['cross-1']
    # Default-grid:
    # cross-1: 1~5
    # cross-2: 1~5
    # diamond-1: 1~5
    # diamond-2: 4~5
    # diamond-3: 4~5
    # x-1: 2~5
    
    import json

    classic_shapes = None
    # Assuming you have a file named 'data.json'
    with open("../assets/data/Polyminoes/default_shapes.json", 'r') as f:
        classic_shapes = json.load(f)

    raw_grid = classic_shapes['diamond-3']
    test_grid = generate_grid(raw_grid = raw_grid)
    cand_shapes, cand_shape_names = get_polyminoes_by_size([5, 4])
    # UPDATE YOUR SELECTED ~MINOES!
    all_cand_shapes = dict()
    for idx, shape in enumerate(cand_shapes):
        new_shapes = all_possible_rotations(shape, cand_shape_names[idx])
        
        for shape_name, new_shape in new_shapes.items():
            all_cand_shapes[shape_name] = new_shape
    
    test_grid['cand_shapes'] = all_cand_shapes
    dummy_grid = Grid(test_grid)
    result_coords, result_shape_name = dummy_grid.solve()

Found!
[[[5, 8], [6, 8], [7, 8], [8, 8]], [[5, 4], [3, 3], [4, 3], [5, 3]], [[5, 1], [6, 1], [5, 0], [6, 0]], [[7, 3], [7, 2], [8, 2], [7, 1]], [[5, 11], [6, 11], [6, 10], [7, 10]], [[0, 6], [0, 5], [1, 5], [2, 5], [1, 4]], [[4, 10], [4, 9], [4, 8], [4, 7], [4, 6]], [[5, 10], [5, 9], [6, 9], [7, 9], [8, 9]], [[5, 7], [6, 7], [7, 7], [7, 6], [8, 6]], [[10, 6], [11, 6], [10, 5], [11, 5], [10, 4]], [[3, 7], [1, 6], [2, 6], [3, 6], [3, 5]], [[6, 4], [7, 4], [8, 4], [6, 3], [8, 3]], [[7, 5], [8, 5], [9, 5], [9, 4], [9, 3]], [[3, 9], [2, 8], [3, 8], [1, 7], [2, 7]], [[9, 8], [8, 7], [9, 7], [10, 7], [9, 6]], [[3, 2], [4, 2], [5, 2], [6, 2], [4, 1]], [[4, 5], [2, 4], [3, 4], [4, 4], [2, 3]]]


## Parse to std format

In [34]:
def parse_to_puzzler_std_format(result_coords, result_shape_name):
    result_str = ""
    for coords, shape in zip(result_coords, result_shape_name):
        row = " ".join(f"{x},{y}" for x, y in coords)
        result_str += f"{row} {shape}\n"
    return result_str

if __name__ == "__main__":
    result_str = parse_to_puzzler_std_format(result_coords, result_shape_name)
    print(result_str)

8,13 8,12 I2
9,7 10,7 11,7 I3
13,6 12,5 13,5 V3
8,3 8,2 8,1 8,0 I4
3,6 1,5 2,5 3,5 L4
6,7 7,7 6,6 7,6 O4
5,5 5,4 6,4 5,3 T4
8,5 7,4 8,4 7,3 Z4
7,13 6,12 7,12 7,11 8,11 F
8,8 9,8 10,8 11,8 12,8 I
5,13 6,13 5,12 5,11 5,10 L
5,9 6,9 7,9 4,8 5,8 N
9,6 10,6 9,5 10,5 11,5 P
0,7 0,6 1,6 2,6 0,5 T
5,1 7,1 5,0 6,0 7,0 U
2,7 3,7 4,7 4,6 4,5 V
13,8 12,7 13,7 11,6 12,6 W
6,3 5,2 6,2 7,2 6,1 X
0,8 1,8 2,8 3,8 1,7 Y
6,11 6,10 7,10 8,10 8,9 Z

